In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
from datetime import datetime



df['Date'] = pd.to_datetime(df['Date'])

features = pd.get_dummies(df[['Day_of_week', 'Number_of_signals', 'Average_speed', 'Vehicle_count', 'Average_time_per_signal']], 
                          columns=['Day_of_week'], drop_first=False)
target = df['Time_to_reach']  

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


with open('traffic_model.pkl', 'wb') as f:
    pickle.dump(model, f)



In [ ]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

with open('traffic_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

def prepare_features(source, destination, date, time):
    day_of_week = date.strftime('%A')
    
    number_of_signals = df[(df['Source'] == source) & (df['Destination'] == destination)]['Number_of_signals'].mean()
    average_speed = df[(df['Source'] == source) & (df['Destination'] == destination)]['Average_speed'].mean()
    vehicle_count = df[(df['Source'] == source) & (df['Destination'] == destination)]['Vehicle_count'].mean()
    average_time_per_signal = df[(df['Source'] == source) & (df['Destination'] == destination)]['Average_time_per_signal'].mean()
    
    features = {
        'Number_of_signals': number_of_signals,
        'Average_speed': average_speed,
        'Vehicle_count': vehicle_count,
        'Average_time_per_signal': average_time_per_signal
    }
    
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        features['Day_of_week_' + day] = 1 if day == day_of_week else 0
    
    features_df = pd.DataFrame([features])
    
    return features_df

def predict_time(source, destination, date, time):
    features_df = prepare_features(source, destination, date, time)
    
    predicted_time = loaded_model.predict(features_df)
    
    return predicted_time[0]

